In [37]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [38]:
# Import GYM Stuff
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

# Import helpers
import numpy as np
import random
import os

# Import stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [39]:
Discrete(3)

Discrete(3)

In [40]:
#checkout values of discrete space
Discrete(3).sample()

1

In [41]:
Box(0,1,shape=(3,3)).sample()

#Box(0,1,shape=(3,).sample() # if only 3 values

array([[0.318348  , 0.9790222 , 0.903918  ],
       [0.8519846 , 0.8782003 , 0.25811505],
       [0.2902012 , 0.725846  , 0.15990888]], dtype=float32)

In [42]:
#Tuple are not supported, but can be generated - does not work

In [43]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 22, 137, 187],
       [ 17,  13, 113],
       [169, 236,  77]])

In [44]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(1, array([74.43873], dtype=float32))

In [45]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([60.535774], dtype=float32))])

In [46]:
MultiBinary(4).sample()

array([1, 0, 1, 1], dtype=int8)

In [47]:
MultiDiscrete([5,2,2]).sample()

array([2, 0, 0], dtype=int64)

# 3. Building an Environment

In [48]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)  #tap/button up-down-unchanged
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))  #
        # Set start temp
        self.state = 38 + random.randint(-3,3)  #shower start 38 +/- 3
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 - no change
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [49]:
env=ShowerEnv()

c:\Users\PeterMeyerDr\anaconda3\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [50]:
#check it out - repeat
env.observation_space.sample()

array([53.40703], dtype=float32)

In [51]:
env.observation_space.sample()

array([75.91612], dtype=float32)

In [52]:
env.reset()

array([38.])

In [53]:
from stable_baselines3.common.env_checker import check_env

In [56]:
#check_env(env, warn=True)

# 4. Test Environment

In [57]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-50
Episode:2 Score:-26
Episode:3 Score:4
Episode:4 Score:-20
Episode:5 Score:4


In [58]:
env.close()

# 5. Train Model

In [59]:
log_path = os.path.join('Training', 'Logs')

In [60]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [68]:
model.learn(total_timesteps=400000)  #play with value

Logging to Training\Logs\PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -27.2    |
| time/              |          |
|    fps             | 375      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -25          |
| time/                   |              |
|    fps                  | 206          |
|    iterations           | 2            |
|    time_elapsed         | 19           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0077010524 |
|    clip_fraction        | 0.0858       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.926       |
|    explained_variance   |

# 6. Save Model

In [73]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [74]:
model.save(shower_path)
# model.save('PPO')

In [75]:
# del and restore
del model
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [76]:
#evaluate_policy(model, env, n_eval_episodes=10, render=True)
evaluate_policy(model, env, n_eval_episodes=10)

(-12.0, 58.787753826796276)